<a href="https://colab.research.google.com/github/RoloNatt/English-to-Hindi-Translation/blob/main/English_to_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

<ipython-input-3-096836cbf52f>:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [4]:
lines = pd.read_csv('small_data.csv')
lines = lines[:5000]

In [5]:
lines.head()

,hindi,english
0,तो लघुतम संपवर्तक होगा—क्यूंकी इन,So the least common multiple is going to be - - because these
1,इब्राहिम ने कहा क्या मुझे ख़ुशख़बरी (बेटा होने की) देते हो जब मुझे बुढ़ापा छा गया,"He said, “Do you bring me good news, when old age has overtaken me? What good news do you bring? ”"
2,प्रयोग के लिए इकाई जब तापमान दिखाया जा रहा हो.,The unit to use when showing temperatures.
3,और जिन्हें तुम पूजते हो मैं उनका पूजने वाला नहीं,"Nor am I worshiping what you have worshipped,"
4,सिनसिनाटी मास्टर्स,1989 Cincinnati Open 1989


In [6]:
lines['english']=lines['english'].apply(lambda x: str(x))
lines['hindi']=lines['hindi'].apply(lambda x: str(x))
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())

In [7]:
lines['hindi'][0]

'तो लघुतम संपवर्तक होगा—क्यूंकी इन'

In [8]:
# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub("'", '', x))

In [9]:
lines.head()

,hindi,english
0,तो लघुतम संपवर्तक होगा—क्यूंकी इन,so the least common multiple is going to be - - because these
1,इब्राहिम ने कहा क्या मुझे ख़ुशख़बरी (बेटा होने की) देते हो जब मुझे बुढ़ापा छा गया,"he said, “do you bring me good news, when old age has overtaken me? what good news do you bring? ”"
2,प्रयोग के लिए इकाई जब तापमान दिखाया जा रहा हो.,the unit to use when showing temperatures.
3,और जिन्हें तुम पूजते हो मैं उनका पूजने वाला नहीं,"nor am i worshiping what you have worshipped,"
4,सिनसिनाटी मास्टर्स,1989 cincinnati open 1989


In [10]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [11]:
lines.head()

,hindi,english
0,तो लघुतम संपवर्तक होगा—क्यूंकी इन,so the least common multiple is going to be because these
1,इब्राहिम ने कहा क्या मुझे ख़ुशख़बरी बेटा होने की देते हो जब मुझे बुढ़ापा छा गया,he said “do you bring me good news when old age has overtaken me what good news do you bring ”
2,प्रयोग के लिए इकाई जब तापमान दिखाया जा रहा हो,the unit to use when showing temperatures
3,और जिन्हें तुम पूजते हो मैं उनका पूजने वाला नहीं,nor am i worshiping what you have worshipped
4,सिनसिनाटी मास्टर्स,1989 cincinnati open 1989


In [12]:
remove_digits = str.maketrans('', '', digits)

In [13]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [14]:
a = lines['english'][0].translate(remove_digits)

In [15]:
a

'so the least common multiple is going to be   because these'

In [16]:
a.strip()

'so the least common multiple is going to be   because these'

In [17]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [18]:
lines['english'][0]

'so the least common multiple is going to be because these'

In [19]:
# Add start and end tokens to target sequences
lines['hindi'] = lines['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [20]:
lines['hindi'][0]

'START_ तो लघुतम संपवर्तक होगा—क्यूंकी इन _END'

In [21]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [22]:
lines.head()

,hindi,english
0,START_ तो लघुतम संपवर्तक होगा—क्यूंकी इन _END,so the least common multiple is going to be because these
1,START_ इब्राहिम ने कहा क्या मुझे ख़ुशख़बरी बेटा होने की देते हो जब मुझे बुढ़ापा छा गया _END,he said “do you bring me good news when old age has overtaken me what good news do you bring ”
2,START_ प्रयोग के लिए इकाई जब तापमान दिखाया जा रहा हो _END,the unit to use when showing temperatures
3,START_ और जिन्हें तुम पूजते हो मैं उनका पूजने वाला नहीं _END,nor am i worshiping what you have worshipped
4,START_ सिनसिनाटी मास्टर्स _END,cincinnati open


In [23]:
lines['length_eng']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin']=lines['hindi'].apply(lambda x:len(x.split(" ")))

In [24]:
lines.head()
lines[lines['length_eng']>30].shape

(450, 4)

In [25]:
lines=lines[lines['length_eng']<=20]
lines=lines[lines['length_hin']<=20]

In [26]:
print("maximum length of Hindi Sentence ",max(lines['length_hin']))
print("maximum length of English Sentence ",max(lines['length_eng']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [27]:
max_length_src=max(lines['length_hin'])
max_length_tar=max(lines['length_eng'])

In [28]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(11026, 13227)

In [29]:
num_decoder_tokens

13227

In [30]:
num_decoder_tokens += 1

In [31]:
num_decoder_tokens

13228

In [32]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [33]:
input_token_index

{'a': 1,
 'aa': 2,
 'aakashdeep': 3,
 'aampapad': 4,
 'aaron': 5,
 'aastuny': 6,
 'abandon': 7,
 'abase': 8,
 'abasements': 9,
 'abated': 10,
 'abating': 11,
 'abdel': 12,
 'abdul': 13,
 'abhi': 14,
 'abhinav': 15,
 'abide': 16,
 'abiding': 17,
 'abidjan': 18,
 'ability': 19,
 'abiword': 20,
 'able': 21,
 'abnormal': 22,
 'abnormally': 23,
 'abode': 24,
 'abomination': 25,
 'about': 26,
 'above': 27,
 'abraham': 28,
 'abroad': 29,
 'abruptly': 30,
 'abscess': 31,
 'absence': 32,
 'absent': 33,
 'absents': 34,
 'absolute': 35,
 'absolutely': 36,
 'absolution': 37,
 'absorb': 38,
 'absorbed': 39,
 'abstract': 40,
 'absurd': 41,
 'abtar': 42,
 'abu': 43,
 'abundant': 44,
 'abuse': 45,
 'academic': 46,
 'academics': 47,
 'academy': 48,
 'acceded': 49,
 'accentuation': 50,
 'accept': 51,
 'acceptable': 52,
 'acceptance': 53,
 'accepted': 54,
 'accepting': 55,
 'accepts': 56,
 'access': 57,
 'accessed': 58,
 'accessibility': 59,
 'accessing': 60,
 'accident': 61,
 'accidents': 62,
 'accompan

In [34]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [35]:
reverse_input_char_index

{1: 'a',
 2: 'aa',
 3: 'aakashdeep',
 4: 'aampapad',
 5: 'aaron',
 6: 'aastuny',
 7: 'abandon',
 8: 'abase',
 9: 'abasements',
 10: 'abated',
 11: 'abating',
 12: 'abdel',
 13: 'abdul',
 14: 'abhi',
 15: 'abhinav',
 16: 'abide',
 17: 'abiding',
 18: 'abidjan',
 19: 'ability',
 20: 'abiword',
 21: 'able',
 22: 'abnormal',
 23: 'abnormally',
 24: 'abode',
 25: 'abomination',
 26: 'about',
 27: 'above',
 28: 'abraham',
 29: 'abroad',
 30: 'abruptly',
 31: 'abscess',
 32: 'absence',
 33: 'absent',
 34: 'absents',
 35: 'absolute',
 36: 'absolutely',
 37: 'absolution',
 38: 'absorb',
 39: 'absorbed',
 40: 'abstract',
 41: 'absurd',
 42: 'abtar',
 43: 'abu',
 44: 'abundant',
 45: 'abuse',
 46: 'academic',
 47: 'academics',
 48: 'academy',
 49: 'acceded',
 50: 'accentuation',
 51: 'accept',
 52: 'acceptable',
 53: 'acceptance',
 54: 'accepted',
 55: 'accepting',
 56: 'accepts',
 57: 'access',
 58: 'accessed',
 59: 'accessibility',
 60: 'accessing',
 61: 'accident',
 62: 'accidents',
 63: 'acco

In [36]:
lines.head(10)

,hindi,english,length_eng,length_hin
0,START_ तो लघुतम संपवर्तक होगा—क्यूंकी इन _END,so the least common multiple is going to be because these,11,7
2,START_ प्रयोग के लिए इकाई जब तापमान दिखाया जा रहा हो _END,the unit to use when showing temperatures,7,12
3,START_ और जिन्हें तुम पूजते हो मैं उनका पूजने वाला नहीं _END,nor am i worshiping what you have worshipped,8,12
4,START_ सिनसिनाटी मास्टर्स _END,cincinnati open,2,4
5,START_ और शैतान तुम्हें रोक न दे निश्चय ही वह तुम्हारा खुला शत्रु है _END,let not satan bar you he is for you a manifest foe,12,15
6,START_ विरूद्धार्थी _END,antonym,1,3
8,START_ विभिन्न औषधी आवृत्ति और हमलों की गंभीरता के आधार पर ली जाती है। _END,different drugs are used depending on the frequency and severity of the attacks,13,15
9,START_ असाई _END,rude,1,3
12,START_ अध्ययन सामग्री के डिस्पैच के सत्र के लिए धारा के लिए स्थिति _END,status of dispatch of study material for stream for the session,11,14
13,START_ भारत का हस्तशिल्प निर्यात _END,indian handicraft industry,3,6


In [37]:
from sklearn.model_selection import train_test_split
X, y = lines['english'], lines['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((2972,), (744,))

In [38]:
X_train

2652    a momentary spasmodic contraction of a muscle fiber
788     emanuele tamponi                                   
2279    and b i don t even hang out with people like that  
3951    douglas devananda                                  
392     hellenic                                           
          ...                                              
1511    tart                                               
1731    for the benefit of prospective entrepreneurs       
1142    renchi raju                                        
4730    lop buri                                           
4289    file s could not be opened s                       
Name: english, Length: 2972, dtype: object

In [39]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [40]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [41]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [42]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [44]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 100

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    3308100     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    3968700     ['input_2[0][0]']                
                                                                                              

In [45]:
model.save('eng-to-hindi.h5')

In [46]:
a, b = next(generate_batch())

In [47]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [48]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=100,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/100


<ipython-input-48-251c55099ab9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


46/46 [==============================] - 38s 503ms/step - loss: 8.1179 - accuracy: 0.1358 - val_loss: 7.3408 - val_accuracy: 0.1434
Epoch 2/100
46/46 [==============================] - 7s 156ms/step - loss: 6.7636 - accuracy: 0.1477 - val_loss: 7.3018 - val_accuracy: 0.1563
Epoch 3/100
46/46 [==============================] - 6s 138ms/step - loss: 6.4377 - accuracy: 0.1572 - val_loss: 7.2683 - val_accuracy: 0.1590
Epoch 4/100
46/46 [==============================] - 7s 153ms/step - loss: 6.2136 - accuracy: 0.1600 - val_loss: 7.3267 - val_accuracy: 0.1595
Epoch 5/100
46/46 [==============================] - 6s 141ms/step - loss: 6.0360 - accuracy: 0.1647 - val_loss: 7.3973 - val_accuracy: 0.1576
Epoch 6/100
46/46 [==============================] - 8s 167ms/step - loss: 5.8924 - accuracy: 0.1689 - val_loss: 7.4734 - val_accuracy: 0.1578
Epoch 7/100
46/46 [==============================] - 8s 168ms/step - loss: 5.7553 - accuracy: 0.1741 - val_loss: 7.5328 - val_accuracy: 0.1611
Epoch 8/10

In [49]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [50]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [51]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 42ms/step
Input English sentence: a momentary spasmodic contraction of a muscle fiber
Actual Hindi Translation:  मांसपेशी फाइबर का एक क्षणिक अकड़नेवाला संकुचन। 
Predicted Hindi Translation:  मांसपेशी फाइबर का एक क्षणिक अकड़नेवाला संकुचन। 


In [53]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: emanuele tamponi
Actual Hindi Translation:  एमेनुएल ताम्पोनी 
Predicted Hindi Translation:  एमेनुएल ताम्पोनी 


In [54]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 35ms/step
Input English sentence: and b i don t even hang out with people like that
Actual Hindi Translation:  और दूसरी बातः मैं ऐसे लोगों से दोस्ती भी नहीं रखती। 
Predicted Hindi Translation:  और दूसरी बातः मैं ऐसे लोगों से दोस्ती भी नहीं र
